In [1]:
import findspark
findspark.init('/home/gerardo-rodriguez/spark-4.0.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/26 10:58:37 WARN Utils: Your hostname, Lanz-Lenovo, resolves to a loopback address: 127.0.1.1; using 192.168.1.145 instead (on interface wlp2s0)
25/08/26 10:58:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/26 10:58:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [612]:
df = spark.read.csv('../Create_ratings/netflix_users.csv', header=True, inferSchema=True)

In [613]:
df.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Subscription_Type: string (nullable = true)
 |-- Watch_Time_Hours: double (nullable = true)
 |-- Favorite_Genre: string (nullable = true)
 |-- Last_Login: date (nullable = true)



# Create Column Churn

In [614]:
from pyspark.sql.functions import col, datediff, current_date, when

In [615]:
df_churn = df.withColumn('churn', when((col('Watch_Time_Hours') < 300) & (datediff(current_date(), col('Last_Login')) > 60), 1).otherwise(0))

In [616]:
df_ml = df_churn.withColumn('days_login', datediff(current_date(), col('Last_Login')))
df_ml.show(5)

+-------+--------------+---+-------+-----------------+----------------+--------------+----------+-----+----------+
|User_ID|          Name|Age|Country|Subscription_Type|Watch_Time_Hours|Favorite_Genre|Last_Login|churn|days_login|
+-------+--------------+---+-------+-----------------+----------------+--------------+----------+-----+----------+
|      1|James Martinez| 18| France|          Premium|           80.26|         Drama|2024-05-12|    1|       471|
|      2|   John Miller| 23|    USA|          Premium|          321.75|        Sci-Fi|2025-02-05|    0|       202|
|      3|    Emma Davis| 60|     UK|            Basic|           35.89|        Comedy|2025-01-24|    1|       214|
|      4|   Emma Miller| 44|    USA|          Premium|          261.56|   Documentary|2024-03-25|    1|       519|
|      5|    Jane Smith| 68|    USA|         Standard|           909.3|         Drama|2025-01-14|    0|       224|
+-------+--------------+---+-------+-----------------+----------------+---------

# Classification Model for churn

In [617]:
from pyspark.ml.classification import (LogisticRegression, RandomForestClassifier)
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [618]:
indexer = StringIndexer(inputCols=['Subscription_Type', 'Country'], outputCols=['subscription_indexed', 'country_indexed'])
indexed = indexer.fit(df_ml)

In [619]:
df_indexed = indexed.transform(df_ml)

In [620]:
df_indexed.columns

['User_ID',
 'Name',
 'Age',
 'Country',
 'Subscription_Type',
 'Watch_Time_Hours',
 'Favorite_Genre',
 'Last_Login',
 'churn',
 'days_login',
 'subscription_indexed',
 'country_indexed']

## Seleccion de variables 

Para la predicción de abandono (churn), se seleccionaron las siguientes variables por su relevancia potencial en el comportamiento del usuario:

Edad (Age): Los usuarios más jóvenes, al estar más familiarizados con la tecnología y con mayor disposición a explorar nuevas plataformas de streaming, presentan una mayor probabilidad de migrar. En contraste, los usuarios de mayor edad tienden a mantener hábitos más estables, lo que podría reflejarse en una menor tasa de abandono.

Horas de visualización (Watch_Time_Hours): Un mayor tiempo de visualización es un fuerte indicador de compromiso con la plataforma. Si el usuario encuentra contenido de interés y dedica muchas horas, la probabilidad de abandono disminuye significativamente.

Días desde el último acceso (Days_Login): La inactividad prolongada es una señal directa de desinterés o desconexión con el servicio. Usuarios que no han ingresado recientemente son más propensos a cancelar su suscripción.

Tipo de suscripción (Subscription_Type): Los usuarios con planes de mayor costo (como Premium) suelen compartir la cuenta con familiares o amigos. Este factor reduce la probabilidad de abandono, ya que la decisión involucra a más personas. En cambio, los planes básicos, generalmente individuales, presentan mayor volatilidad.

País (Country): La situación económica de cada país influye en la capacidad de pago. En regiones donde el valor del dólar representa un costo elevado, los usuarios pueden considerar la suscripción como un gasto prescindible, aumentando el riesgo de abandono.

## Features Selection

For churn prediction, the following variables were selected due to their potential relevance to user behavior:

Age: Younger users, being more familiar with technology and more inclined to explore alternative streaming platforms, may have a higher probability of leaving. In contrast, older users often maintain more stable habits, which may translate into lower churn rates.

Watch Time (Watch_Time_Hours): Higher watch time is a strong indicator of user engagement. If users find content of interest and dedicate significant hours to watching, the likelihood of churn decreases.

Days Since Last Login (Days_Login): Extended inactivity is a direct signal of disengagement. Users who have not logged in recently are more likely to cancel their subscription.

Subscription Type (Subscription_Type): Users with higher-tier plans (e.g., Premium) often share accounts with family or friends. This shared dependency lowers the probability of churn, as the decision to cancel involves multiple people. Conversely, basic plans, usually for individual use, tend to be more volatile.

Country: Economic conditions in each country affect users’ ability to pay. In regions where the dollar represents a relatively high cost, users may perceive the subscription as a non-essential expense, increasing churn risk.

In [621]:
asembler = VectorAssembler(inputCols=['Age',
 'Watch_Time_Hours',
 'days_login',
 'subscription_indexed',
 'country_indexed'], outputCol='features')

In [622]:
df_assembled = asembler.transform(df_indexed)

In [623]:
df_assembled.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Subscription_Type: string (nullable = true)
 |-- Watch_Time_Hours: double (nullable = true)
 |-- Favorite_Genre: string (nullable = true)
 |-- Last_Login: date (nullable = true)
 |-- churn: integer (nullable = false)
 |-- days_login: integer (nullable = true)
 |-- subscription_indexed: double (nullable = false)
 |-- country_indexed: double (nullable = false)
 |-- features: vector (nullable = true)



In [624]:
train_data, test_data = df_assembled.randomSplit([0.8, 0.2], seed=1)

### Logistic Regression

In [625]:
logistic = LogisticRegression(featuresCol='features', labelCol='churn')
model = logistic.fit(train_data)

In [626]:
prediction = model.transform(test_data)

In [627]:
prediction.select(['churn', 'prediction']).show()

+-----+----------+
|churn|prediction|
+-----+----------+
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows


In [628]:
eva_log = BinaryClassificationEvaluator(labelCol='churn')
pred_log = eva_log.evaluate(prediction)

In [629]:
print("Evaluation Logistic regression")
pred_log

Evaluation Logistic regression


0.999999415457866

### Random Forest

In [630]:
rf = RandomForestClassifier(featuresCol='features', labelCol='churn', maxDepth=20, numTrees=200, seed=1)
model_rf = rf.fit(train_data)

25/08/26 13:06:40 WARN DAGScheduler: Broadcasting large task binary with size 1056.2 KiB
25/08/26 13:06:41 WARN DAGScheduler: Broadcasting large task binary with size 1535.9 KiB
25/08/26 13:06:42 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
25/08/26 13:06:43 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
25/08/26 13:06:44 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB
25/08/26 13:06:45 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
25/08/26 13:06:46 WARN DAGScheduler: Broadcasting large task binary with size 4.9 MiB
25/08/26 13:06:47 WARN DAGScheduler: Broadcasting large task binary with size 5.4 MiB
25/08/26 13:06:48 WARN DAGScheduler: Broadcasting large task binary with size 5.8 MiB
25/08/26 13:06:49 WARN DAGScheduler: Broadcasting large task binary with size 5.8 MiB
25/08/26 13:06:50 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
25/08/26 13:06:50 WARN DAGScheduler: Broadcastin

In [631]:
predict_rf = model_rf.transform(test_data)
predict_rf.select(['churn', 'prediction']).show(5)

25/08/26 13:06:55 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


+-----+----------+
|churn|prediction|
+-----+----------+
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 5 rows


In [632]:
eva_accuracy = MulticlassClassificationEvaluator(labelCol='churn', metricName='accuracy')
eva_precision = MulticlassClassificationEvaluator(labelCol='churn', metricName='weightedPrecision')
eva_recall = MulticlassClassificationEvaluator(labelCol='churn', metricName='weightedRecall')
eva_f1 = MulticlassClassificationEvaluator(labelCol='churn', metricName='f1')

accuracy = eva_accuracy.evaluate(predict_rf)
precision = eva_precision.evaluate(predict_rf)
recall = eva_recall.evaluate(predict_rf)
f1 = eva_f1.evaluate(predict_rf)

25/08/26 13:06:58 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
25/08/26 13:06:59 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
25/08/26 13:06:59 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
25/08/26 13:07:00 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


In [634]:
print("Métricas Random Forest")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

Métricas Random Forest
Accuracy : 0.9954
Precision: 0.9955
Recall   : 0.9954
F1 Score : 0.9954


## Evaluation Metrics – Random Forest (Churn Prediction)

| Metric     | Value  |
|------------|--------|
| Accuracy   | 0.9954 |
| Precision  | 0.9960 |
| Recall     | 0.9940 |
| F1 Score   | 0.9950 |


# Linear Model

In [495]:
from pyspark.ml.regression import LinearRegression

In [496]:
df_show = spark.read.csv('../Create_ratings/netflix_titles.csv', header=True, inferSchema=True)
df_show = df_show.select(['show_id',
                     'type',
                     'title',
                     'director',
                     'country',
                     'date_added',
                     'release_year',
                     'rating',
                     'duration',
                     'listed_in',
                     'description']).filter(df_show['type'] == 'Movie')

In [497]:
from pyspark.sql.functions import rand

In [498]:
df_show = df_show.withColumn('rate', rand() * 1)

In [499]:
df_show = df_show.dropna('any')
df_show.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)
 |-- rate: double (nullable = false)



In [500]:
df_show.columns

['show_id',
 'type',
 'title',
 'director',
 'country',
 'date_added',
 'release_year',
 'rating',
 'duration',
 'listed_in',
 'description',
 'rate']

In [501]:
df_train = df_show.select([
    'title',
 'director',
 'release_year',
 'duration',
 'rating',
 'rate'])

df_train.printSchema()

root
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rate: double (nullable = false)



In [502]:
indexer_director = StringIndexer(inputCol='director', outputCol='director_indexed')
indexer_rating = StringIndexer(inputCol="rating", outputCol="rating_indexed", handleInvalid="keep")

In [526]:
from pyspark.sql.functions import year, month, day, regexp_replace, when

In [550]:
def preprocess(df):
    df = df.withColumn("year", col("release_year"))

    df = df.withColumn(
        "duration_num",
        regexp_replace(col("duration"), " min", "").cast("int")
    )

    return df
    

In [551]:
df_prepared = preprocess(df_train)

In [564]:
assembler_rf = VectorAssembler(inputCols=[ 'rating_indexed', 'director_indexed'], outputCol='features')

In [565]:
train_data, test_data = df_rf.randomSplit([0.8, 0.2])

In [566]:
lin_reg = LinearRegression(featuresCol='features', labelCol='rate', maxIter=150, regParam=20)

In [567]:
from pyspark.ml.pipeline import Pipeline

In [568]:
pipeline = Pipeline(stages=[indexer_director, indexer_rating, assembler_rf, lin_reg])

In [569]:
model = pipeline.fit(df_prepared)

In [570]:
predictions = model.transform(df_prepared)
predictions.select("title", "rate", "prediction").show(10, truncate=False)
df_prepared.printSchema()

+--------------------+-------------------+------------------+
|title               |rate               |prediction        |
+--------------------+-------------------+------------------+
|Dick Johnson Is Dead|0.7175281420390734 |0.5039205129779584|
|Sankofa             |0.09022270731839044|0.5037995371653556|
|The Starling        |0.10834694478589335|0.5038816017447478|
|Je Suis Karl        |0.20763055944118902|0.5038149922927476|
|Jeans               |0.37387273012813194|0.5037882652502326|
|Grown Ups           |0.8099056034834983 |0.5039901042219693|
|Dark Skies          |0.8861794114264411 |0.5039730949476887|
|Paranoia            |0.8079191014047463 |0.50399162958836  |
|Birth of the Dragon |0.749882942178189  |0.5039397095700822|
|Jaws                |0.9201593787689151 |0.5040271259823841|
+--------------------+-------------------+------------------+
only showing top 10 rows
root
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- release_year: string

In [571]:
from pyspark.ml.evaluation import RegressionEvaluator

## Nota sobre la calidad del modelo

Es importante recalcar que las métricas de evaluación (RMSE, MAE y R²) presentan valores bajos debido a que los datos utilizados fueron generados de manera completamente aleatoria. Esto significa que no existe una relación lógica o causal entre las variables predictoras y la variable objetivo (rate). En consecuencia, el modelo no puede encontrar patrones reales y sus predicciones no son representativas de un escenario práctico. El objetivo de este ejercicio es únicamente demostrar la aplicación de técnicas de Machine Learning con PySpark y no obtener un modelo predictivo útil en la realidad.

It is important to highlight that the evaluation metrics (RMSE, MAE, and R²) show poor values because the dataset was entirely randomly generated. This means that there is no logical or causal relationship between the predictor variables and the target variable (rate). As a result, the model cannot identify real patterns, and its predictions are not representative of a practical scenario. The purpose of this exercise is solely to demonstrate the application of Machine Learning techniques with PySpark, rather than to build a useful predictive model in reality.

In [635]:
evaluator_rmse = RegressionEvaluator(labelCol="rate", predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol="rate", predictionCol="prediction", metricName="r2")
evaluator_mae = RegressionEvaluator(labelCol="rate", predictionCol="prediction", metricName="mae")

rmse = evaluator_rmse.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)
mae = evaluator_mae.evaluate(predictions)

In [636]:
print("Métricas de Evaluación:")
print(f"RMSE: {rmse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R²:   {r2:.4f}")

Métricas de Evaluación:
RMSE: 0.2912
MAE:  0.2530
R²:   0.0000


# Evaluation Metrics – Linear Regression

| Metric     | Value  |
|------------|--------|
| RMSE   | 0.2912 |
| r2  | 0.2530 |
| mae     | 0.0000 |



# Kmeans

In [573]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScaler

In [574]:
df_users = spark.read.csv('../Create_ratings/netflix_users.csv', header=True, inferSchema=True)

In [575]:
df_users.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Subscription_Type: string (nullable = true)
 |-- Watch_Time_Hours: double (nullable = true)
 |-- Favorite_Genre: string (nullable = true)
 |-- Last_Login: date (nullable = true)



In [576]:
df_users.columns

['User_ID',
 'Name',
 'Age',
 'Country',
 'Subscription_Type',
 'Watch_Time_Hours',
 'Favorite_Genre',
 'Last_Login']

In [577]:
indexer_country = StringIndexer(inputCol='Country', outputCol='country_indexed')
indexer_sub = StringIndexer(inputCol='Subscription_Type', outputCol='sub_indexed')

In [578]:
assembler_k = VectorAssembler(inputCols=[
'country_indexed',
 'Watch_Time_Hours',
 ], outputCol='features')

In [579]:
scale = StandardScaler(inputCol='features', outputCol='features_scale')

In [594]:
val = [x for x in range(2,11)]

for i in val:
    k_mean = KMeans(k=i, featuresCol='features_scale', seed=1)
    pipeline_km = Pipeline(stages=[indexer_country, indexer_sub, assembler_k, scale, k_mean])
    model = pipeline_km.fit(df_users)
    predic = model.transform(df_users)
    eva_clu = ClusteringEvaluator(featuresCol='features_scale')

    print(f'prediction with k = {i}')
    print(eva_clu.evaluate(predic))
    

prediction with k = 2
0.5034836601134689
prediction with k = 3
0.5426448158943118
prediction with k = 4
0.5951915739690253
prediction with k = 5
0.5581826520502499
prediction with k = 6
0.5442801849133799
prediction with k = 7
0.5072823778367624
prediction with k = 8
0.532688349580143
prediction with k = 9
0.5212583263043946
prediction with k = 10
0.5218923456161457


In [596]:
k_mean = KMeans(k=4, featuresCol='features_scale', seed=1)

In [597]:
pipeline_km = Pipeline(stages=[indexer_country, indexer_sub, assembler_k, scale, k_mean])

In [598]:
model_k = pipeline_km.fit(df_users)

In [599]:
prediction_k = model_k.transform(df_users)

In [600]:
prediction_k.select('Age', 'Watch_Time_Hours', 'prediction').show(5)

+---+----------------+----------+
|Age|Watch_Time_Hours|prediction|
+---+----------------+----------+
| 18|           80.26|         3|
| 23|          321.75|         0|
| 60|           35.89|         0|
| 44|          261.56|         0|
| 68|           909.3|         1|
+---+----------------+----------+
only showing top 5 rows


In [601]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [602]:
eva_clu = ClusteringEvaluator(featuresCol='features_scale')

In [603]:
print('eva of kmean 3')
eva_clu.evaluate(prediction_k)

eva of kmean 3


0.5951915739690253

In [609]:
from pyspark.sql.functions import avg

In [610]:
prediction_k.groupBy('prediction').agg(
    avg('Age'),
    avg('Watch_Time_Hours')
).show()

+----------+------------------+---------------------+
|prediction|          avg(Age)|avg(Watch_Time_Hours)|
+----------+------------------+---------------------+
|         1| 46.54018632559608|    745.0765687667777|
|         3| 46.70972471086496|   255.25532008470475|
|         2| 46.51924005796168|    748.6533633875353|
|         0|46.169225898369476|   249.52558018046565|
+----------+------------------+---------------------+



# Evaluation Groups – Kmeans


|prediction|          avg(Age)|avg(Watch_Time_Hours)|
|----------|------------------|---------------------|
|         1| 46.54018632559608|    745.0765687667777|
|         3| 46.70972471086496|   255.25532008470475|
|         2| 46.51924005796168|    748.6533633875353|
|         0|46.169225898369476|   249.52558018046565|